In [1]:
#r "nuget: Azure.AI.Projects, 1.0.0-beta.2"

Installed Packages Azure.AI.Projects, 1.0.0-beta.2

In [2]:
var connectionString = "Your Azure AI Agent Service Connection String";

In [3]:
#r "nuget: Azure.Identity, 1.13.1"

Installed Packages Azure.Identity, 1.13.1

In [4]:

using Azure.Core;

using Azure.Identity;

using Azure.AI.Projects;

In [5]:
AgentsClient client = new AgentsClient(connectionString, new DefaultAzureCredential());

In [8]:
Azure.Response<AgentFile> uploadAgentFileResponse = await client.UploadFileAsync(
            filePath: "./data/intro_rag.md",
            purpose: AgentFilePurpose.Agents);

In [9]:
AgentFile uploadedAgentFile = uploadAgentFileResponse.Value;

In [10]:
VectorStore vectorStore = await client.CreateVectorStoreAsync(
            fileIds:  new List<string> { uploadedAgentFile.Id },
            name: "my_vector_store");

In [11]:
FileSearchToolResource fileSearchToolResource = new FileSearchToolResource();
        fileSearchToolResource.VectorStoreIds.Add(vectorStore.Id);

In [12]:
Azure.Response<Agent> agentResponse = await client.CreateAgentAsync(
                model: "gpt-4o-mini",
                name: "RAG Agent",
                instructions: "You are a helpful agent that can help fetch data from files you know about.",
                tools: new List<ToolDefinition> { new FileSearchToolDefinition() },
                toolResources: new ToolResources() { FileSearch = fileSearchToolResource });
Agent agent = agentResponse.Value;

In [13]:
Azure.Response<AgentThread> threadResponse = await client.CreateThreadAsync();
AgentThread thread = threadResponse.Value;

In [14]:
Azure.Response<ThreadMessage> messageResponse = await client.CreateMessageAsync(
            thread.Id,
            MessageRole.User,
            "Can you introduce GraphRAG?");
ThreadMessage message = messageResponse.Value;

In [15]:
Azure.Response<ThreadRun> runResponse = await client.CreateRunAsync(thread, agent);

        do
        {
            await Task.Delay(TimeSpan.FromMilliseconds(500));
            runResponse = await client.GetRunAsync(thread.Id, runResponse.Value.Id);
        }
        while (runResponse.Value.Status == RunStatus.Queued
            || runResponse.Value.Status == RunStatus.InProgress);

Azure.Response<PageableList<ThreadMessage>> afterRunMessagesResponse
            = await client.GetMessagesAsync(thread.Id);
IReadOnlyList<ThreadMessage> messages = afterRunMessagesResponse.Value.Data;

In [16]:
foreach (ThreadMessage threadMessage in messages)
{
            Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
            foreach (MessageContent contentItem in threadMessage.ContentItems)
            {
                if (contentItem is MessageTextContent textItem)
                {
                    Console.Write(textItem.Text);
                }
                else if (contentItem is MessageImageFileContent imageFileItem)
                {
                    Console.Write($"<image from ID: {imageFileItem.FileId}");
                }
                Console.WriteLine();
            }
}

2024-12-30 03:19:51 -  assistant: GraphRAG is an AI-based tool designed for content interpretation and search capabilities. It utilizes large language models (LLMs) to parse data and construct knowledge graphs, enabling it to answer user questions based on a user-provided private dataset. 

### Intended Use
GraphRAG is particularly valuable in scenarios requiring critical information discovery and analysis, especially when:
- The necessary insights span multiple documents and contain noisy or misleading information.
- Questions are abstract or thematic, transcending what the underlying data can answer directly.
- Users are trained in analytic and critical reasoning and can verify and augment GraphRAG's responses.

GraphRAG is tailored for deployment with a domain-specific corpus of text data. It does not collect user data, although users should verify the data privacy policies of the LLM used to implement GraphRAG.

### Evaluation Metrics
GraphRAG has undergone various evaluations focu